In [1]:
# Pulled USGS data to DSS via DSSVue plugin.
# Open DSS file and read data into pandas dataframe for analysis.
# All data in UTC timezone.
from pydsstools.heclib.dss import HecDss
from pydsstools.core import TimeSeriesContainer,UNDEFINED
import pandas as pd

dss = HecDss.Open(r"Z:\py\usgs_gage_shp_to_dss\temp.dss")


In [2]:
dss.getPathnameList("")

['/FLAT FORK CK AT SH 7 NR CENTER, TX/08023050/STAGE/01JAN2023/1DAY/USGS/',
 '/SABINE RV AT LOGANSPORT, LA/08022500/STAGE/01JAN2010/1DAY/USGS/',
 '/TENAHA CK NR SHELBYVILLE, TX/08023200/FLOW/01JAN1978/1DAY/USGS/',
 '/TENAHA CK NR SHELBYVILLE, TX/08023200/FLOW/01JAN1979/1DAY/USGS/',
 '/TENAHA CK NR SHELBYVILLE, TX/08023200/FLOW/01JAN1980/1DAY/USGS/',
 '/TENAHA CK NR SHELBYVILLE, TX/08023200/FLOW/01JAN1981/1DAY/USGS/',
 '/SABINE RV AT LOGANSPORT, LA/08022500/FLOW/01JAN1903/1DAY/USGS/',
 '/SABINE RV AT LOGANSPORT, LA/08022500/STAGE/01JAN2009/1DAY/USGS/',
 '/SABINE RV AT LOGANSPORT, LA/08022500/FLOW/01JAN1904/1DAY/USGS/',
 '/SABINE RV AT LOGANSPORT, LA/08022500/FLOW/01JAN1905/1DAY/USGS/',
 '/SABINE RV AT LOGANSPORT, LA/08022500/FLOW/01JAN1906/1DAY/USGS/',
 '/SABINE RV AT LOGANSPORT, LA/08022500/FLOW/01JAN1907/1DAY/USGS/',
 '/SABINE RV AT LOGANSPORT, LA/08022500/FLOW/01JAN1908/1DAY/USGS/',
 '/SABINE RV AT LOGANSPORT, LA/08022500/FLOW/01JAN1909/1DAY/USGS/',
 '/SABINE RV AT LOGANSPORT, LA/080

In [4]:
import geopandas as gpd
gdf = gpd.read_file(r"Z:\Z GIS\LWI_CoastWide\Gages_roughtcut.shp", crs="EPSG:4326")
gdf

,Source,SITENO,SITENAME,CATEGORY,AGENCY,LONGDD,LATDD,SITEURL,geometry
0,NWISMapperExport_EastTexas,08023050,"Flat Fork Ck at SH 7 nr Center, TX",ST,USGS,-94.0985528,31.8806861,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.09855 31.88069)
1,NWISMapperExport_EastTexas,08023200,"Tenaha Ck nr Shelbyville, TX",ST,USGS,-94.0840835,31.76572914,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.08408 31.76573)
2,NWISMapperExport_EastTexas,08022500,"Sabine Rv at Logansport, LA",ST,USGS,-94.0063014,31.97238949,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.00630 31.97239)
3,NWISMapperExport_EastTexas,08023080,"Bayou Grand Cane near Stanley, LA",ST,USGS,-93.9411609,31.9627225,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-93.94116 31.96272)
4,NWISMapperExport_EastTexas,08024480,"Palo Gaucho Bayou at FM 2784 nr Milam, TX",ST,USGS,-93.8946944,31.39508333,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-93.89469 31.39508)
...,...,...,...,...,...,...,...,...,...
589,NWISMapperExport_SouthMiss,02480380,"TUXACHANIE CREEK AT BETHEL ROAD NEAR SAUCIER, MS",ST,USGS,-88.9913778,30.63265556,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-88.99138 30.63266)
590,NWISMapperExport_SouthMiss,02480345,"TCHOUTACABOUFFA RIVER TRIB NEAR WHITE PLAINS, MS",ST,USGS,-88.9068417,30.5910972,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-88.90684 30.59110)
591,NWISMapperExport_SouthMiss,02480590,"TCHOUTACABOUFFA RIVER NR D'IBERVILLE, MS",ST,USGS,-88.8933643,30.4760307,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-88.89336 30.47603)
592,NWISMapperExport_SouthMiss,02479000,"PASCAGOULA RIVER AT MERRILL, MS",ST,USGS,-88.7269444,30.97805556,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-88.72694 30.97806)


In [40]:
# for each SITENO in gdf, get start and end data times
for i,row in gdf.iterrows():
    siteno = row['SITENO']
    # print(siteno)
    # get dss pathname
    pathname_pattern = f"/*/{row['SITENO']}/*/*/*/*/"
    path_pattern_list = dss.getPathnameList(pathname_pattern,sort=1)
    # print(path_pattern_list)

    ts = dss.read_ts(f"/{row['SITENAME']}/{row['SITENO']}/STAGE//1DAY/USGS/")
    if ts.pytimes is None:
        # print(f"no data for {siteno}")
        continue
    else: 
        # print (ts.pytimes[0], ts.pytimes[-1])
        duration = ts.pytimes[-1] - ts.pytimes[0]  
        duration_in_s = duration.total_seconds()
        print (duration)
        years = divmod(duration_in_s, 31536000)[0]
        gdf.loc[i,'stage_data_start'] = ts.pytimes[0]
        gdf.loc[i,'stage_data_end'] = ts.pytimes[-1]
        gdf.loc[i,'stage_data_years'] = years

    # get start and end times for pathname dss record
    # # ts = dss.read_ts(pathname,regular=False,window_flag=0)
    # print(ts.times[0], ts.times[-1])
gdf
    

102 days, 0:00:00
8703 days, 0:00:00
9799 days, 0:00:00
1051 days, 0:00:00
102 days, 0:00:00
8703 days, 0:00:00
9799 days, 0:00:00
1051 days, 0:00:00
102 days, 0:00:00
8703 days, 0:00:00
9799 days, 0:00:00
1051 days, 0:00:00


,Source,SITENO,SITENAME,CATEGORY,AGENCY,LONGDD,LATDD,SITEURL,geometry,stage_data_start,stage_data_end,stage_data_duration,stage_data_years
0,NWISMapperExport_EastTexas,08023050,"Flat Fork Ck at SH 7 nr Center, TX",ST,USGS,-94.0985528,31.8806861,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.09855 31.88069),2023-04-20,2023-07-31,2023-04-20,0.0
1,NWISMapperExport_EastTexas,08023200,"Tenaha Ck nr Shelbyville, TX",ST,USGS,-94.0840835,31.76572914,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.08408 31.76573),NaT,NaT,NaT,NaN
2,NWISMapperExport_EastTexas,08022500,"Sabine Rv at Logansport, LA",ST,USGS,-94.0063014,31.97238949,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.00630 31.97239),1999-10-02,2023-07-31,1999-10-02,23.0
3,NWISMapperExport_EastTexas,08023080,"Bayou Grand Cane near Stanley, LA",ST,USGS,-93.9411609,31.9627225,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-93.94116 31.96272),1996-10-01,2023-07-31,1996-10-01,26.0
4,NWISMapperExport_EastTexas,08024480,"Palo Gaucho Bayou at FM 2784 nr Milam, TX",ST,USGS,-93.8946944,31.39508333,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-93.89469 31.39508),NaT,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,NWISMapperExport_SouthMiss,02480380,"TUXACHANIE CREEK AT BETHEL ROAD NEAR SAUCIER, MS",ST,USGS,-88.9913778,30.63265556,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-88.99138 30.63266),NaT,NaT,NaT,NaN
590,NWISMapperExport_SouthMiss,02480345,"TCHOUTACABOUFFA RIVER TRIB NEAR WHITE PLAINS, MS",ST,USGS,-88.9068417,30.5910972,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-88.90684 30.59110),NaT,NaT,NaT,NaN
591,NWISMapperExport_SouthMiss,02480590,"TCHOUTACABOUFFA RIVER NR D'IBERVILLE, MS",ST,USGS,-88.8933643,30.4760307,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-88.89336 30.47603),NaT,NaT,NaT,NaN
592,NWISMapperExport_SouthMiss,02479000,"PASCAGOULA RIVER AT MERRILL, MS",ST,USGS,-88.7269444,30.97805556,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-88.72694 30.97806),NaT,NaT,NaT,NaN


In [41]:
gdf2=gdf[gdf.stage_data_years.notnull()]
gdf2

,Source,SITENO,SITENAME,CATEGORY,AGENCY,LONGDD,LATDD,SITEURL,geometry,stage_data_start,stage_data_end,stage_data_duration,stage_data_years
0,NWISMapperExport_EastTexas,08023050,"Flat Fork Ck at SH 7 nr Center, TX",ST,USGS,-94.0985528,31.8806861,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.09855 31.88069),2023-04-20,2023-07-31,2023-04-20,0.0
2,NWISMapperExport_EastTexas,08022500,"Sabine Rv at Logansport, LA",ST,USGS,-94.0063014,31.97238949,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.00630 31.97239),1999-10-02,2023-07-31,1999-10-02,23.0
3,NWISMapperExport_EastTexas,08023080,"Bayou Grand Cane near Stanley, LA",ST,USGS,-93.9411609,31.9627225,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-93.94116 31.96272),1996-10-01,2023-07-31,1996-10-01,26.0
5,NWISMapperExport_EastTexas,08025240,"Housen Bayou at SH 87 nr Hemphill, TX",ST,USGS,-93.8443472,31.30316389,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-93.84435 31.30316),2020-09-13,2023-07-31,2020-09-13,2.0
23,NWISMapperExport_Louisiana,08023050,"Flat Fork Ck at SH 7 nr Center, TX",ST,USGS,-94.0985528,31.8806861,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.09855 31.88069),2023-04-20,2023-07-31,2023-04-20,0.0
25,NWISMapperExport_Louisiana,08022500,"Sabine Rv at Logansport, LA",ST,USGS,-94.0063014,31.97238949,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.00630 31.97239),1999-10-02,2023-07-31,1999-10-02,23.0
26,NWISMapperExport_Louisiana,08023080,"Bayou Grand Cane near Stanley, LA",ST,USGS,-93.9411609,31.9627225,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-93.94116 31.96272),1996-10-01,2023-07-31,1996-10-01,26.0
32,NWISMapperExport_Louisiana,08025240,"Housen Bayou at SH 87 nr Hemphill, TX",ST,USGS,-93.8443472,31.30316389,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-93.84435 31.30316),2020-09-13,2023-07-31,2020-09-13,2.0
253,NWISMapperExport_SouthEastTexas,08023050,"Flat Fork Ck at SH 7 nr Center, TX",ST,USGS,-94.0985528,31.8806861,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.09855 31.88069),2023-04-20,2023-07-31,2023-04-20,0.0
255,NWISMapperExport_SouthEastTexas,08022500,"Sabine Rv at Logansport, LA",ST,USGS,-94.0063014,31.97238949,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-94.00630 31.97239),1999-10-02,2023-07-31,1999-10-02,23.0


In [37]:
gdf2.to_file('stage.geojson', EPSG=4326, driver='GeoJSON')